# Dataset Generation Part 3: Run UBEM

In this file, we'll run the previously created `cleaned.geojson` through the UBEM.

In [1]:
!pip install geopy requests eppy

In [2]:
import glob
import json
import math
import os
from eppy.modeleditor import IDF
import urllib.request
from pathlib import Path
import subprocess
from datetime import datetime
import requests
import zipfile
from io import BytesIO

# Point this to your installed EnergyPlus IDD
IDF.setiddname("/Applications/EnergyPlus-25-1-0/Energy+.idd")  # ← UPDATE PATH

We'll need to be able to find the weather station to use with each home. So, we'll add that as a feature in the dataset. We expect that each `weather` entry is already located in the `weather/` folder and has the same name as passed into `weather_station`.

In [3]:
def transform_dataset(dataset_folder='dataset', weather_station='KABE'):
  preprocessed_data_set = glob.glob(f'{dataset_folder}/*/preprocessed.json')

  for preprocessed_data in preprocessed_data_set:
    with open(preprocessed_data, 'r') as f:
      data = json.load(f)
    data["weather"] = weather_station
    with open(preprocessed_data, 'w') as f:
      json.dump(data, f)

In [11]:
transform_dataset(dataset_folder='dataset', weather_station='KABE')

We'll also need to convert the `geojson` into an `idf` file.

In [69]:
import json
from datetime import datetime
from io import StringIO
from eppy.modeleditor import IDF
from eppy.iddcurrent import iddcurrent

def generate_idf_from_geojson(geojson: dict, idf_path: str):
    idf = IDF(StringIO("Version,25.1;"))

    # Metadata and simulation setup - VERSION already included in StringIO above
    idf.newidfobject("TIMESTEP", Number_of_Timesteps_per_Hour=4)
    
    # Add Site Location (required)
    idf.newidfobject("SITE:LOCATION",
                     Name="Site Location",
                     Latitude=40.0,
                     Longitude=-75.0,
                     Time_Zone=-5.0,
                     Elevation=200.0)
    
    idf.newidfobject("SIMULATIONCONTROL",
                     Do_Zone_Sizing_Calculation="No",
                     Do_System_Sizing_Calculation="No",
                     Do_Plant_Sizing_Calculation="No",
                     Run_Simulation_for_Weather_File_Run_Periods="Yes",
                     Run_Simulation_for_Sizing_Periods="No")
    
    today = datetime.today()
    idf.newidfobject("RUNPERIOD",
                     Name="RunPeriod1",
                     Begin_Month=today.month,
                     Begin_Day_of_Month=today.day,
                     End_Month=today.month,
                     End_Day_of_Month=today.day,
                     Use_Weather_File_Holidays_and_Special_Days="Yes",
                     Use_Weather_File_Daylight_Saving_Period="Yes",
                     Apply_Weekend_Holiday_Rule="Yes",
                     Use_Weather_File_Rain_Indicators="Yes",
                     Use_Weather_File_Snow_Indicators="Yes")

    # Extract geometry info
    building = next(f for f in geojson["features"] if f["properties"]["type"] == "Building")
    props = building["properties"]
    name = props.get("name", "GeneratedHome")
    floor_area = props.get("floor_area", 100.0)
    stories = props.get("number_of_stories", 1)

    length = width = floor_area ** 0.5
    height = stories * 3

    idf.newidfobject("BUILDING",
                     Name=name,
                     North_Axis=0.0,
                     Terrain="Suburbs",
                     Loads_Convergence_Tolerance_Value=0.04,
                     Temperature_Convergence_Tolerance_Value=0.4,
                     Solar_Distribution="FullExterior",
                     Maximum_Number_of_Warmup_Days=25,
                     Minimum_Number_of_Warmup_Days=6)

    idf.newidfobject("GLOBALGEOMETRYRULES",
                     Starting_Vertex_Position="UpperLeftCorner",
                     Vertex_Entry_Direction="CounterClockWise",
                     Coordinate_System="Relative")

    idf.newidfobject("ZONE", Name="MainZone")

    # Materials
    idf.newidfobject("MATERIAL", Name="Wall Material", Roughness="Rough",
                     Thickness=0.2, Conductivity=0.5, Density=1400,
                     Specific_Heat=1000, Thermal_Absorptance=0.9,
                     Solar_Absorptance=0.7, Visible_Absorptance=0.7)
    idf.newidfobject("MATERIAL", Name="Floor Material", Roughness="Rough",
                     Thickness=0.1, Conductivity=1.0, Density=2000,
                     Specific_Heat=1000, Thermal_Absorptance=0.9,
                     Solar_Absorptance=0.7, Visible_Absorptance=0.7)
    idf.newidfobject("MATERIAL", Name="Roof Material", Roughness="Rough",
                     Thickness=0.15, Conductivity=0.3, Density=800,
                     Specific_Heat=1200, Thermal_Absorptance=0.9,
                     Solar_Absorptance=0.7, Visible_Absorptance=0.7)

    idf.newidfobject("CONSTRUCTION", Name="Wall Construction", Outside_Layer="Wall Material")
    idf.newidfobject("CONSTRUCTION", Name="Floor Construction", Outside_Layer="Floor Material")
    idf.newidfobject("CONSTRUCTION", Name="Roof Construction", Outside_Layer="Roof Material")

    # Surfaces - Fixed vertex order for floor and roof
    surfaces = {
        "South Wall": [(0,0,0), (0,0,height), (length,0,height), (length,0,0)],
        "East Wall":  [(length,0,0), (length,0,height), (length,width,height), (length,width,0)],
        "North Wall": [(length,width,0), (length,width,height), (0,width,height), (0,width,0)],
        "West Wall":  [(0,width,0), (0,width,height), (0,0,height), (0,0,0)],
        "Floor":      [(0,width,0), (length,width,0), (length,0,0), (0,0,0)],  # Fixed order
        "Roof":       [(0,0,height), (length,0,height), (length,width,height), (0,width,height)]  # Fixed order
    }

    for name, verts in surfaces.items():
        surface_type = "Wall" if "Wall" in name else "Roof" if name == "Roof" else "Floor"
        construction = f"{surface_type} Construction"
        sun = "NoSun" if surface_type == "Floor" else "SunExposed"
        wind = "NoWind" if surface_type == "Floor" else "WindExposed"
        outside = "Ground" if surface_type == "Floor" else "Outdoors"

        idf.newidfobject("BUILDINGSURFACE:DETAILED",
                         Name=name,
                         Surface_Type=surface_type,
                         Construction_Name=construction,
                         Zone_Name="MainZone",
                         Outside_Boundary_Condition=outside,
                         Sun_Exposure=sun,
                         Wind_Exposure=wind,
                         Number_of_Vertices=4,
                         Vertex_1_Xcoordinate=verts[0][0], Vertex_1_Ycoordinate=verts[0][1], Vertex_1_Zcoordinate=verts[0][2],
                         Vertex_2_Xcoordinate=verts[1][0], Vertex_2_Ycoordinate=verts[1][1], Vertex_2_Zcoordinate=verts[1][2],
                         Vertex_3_Xcoordinate=verts[2][0], Vertex_3_Ycoordinate=verts[2][1], Vertex_3_Zcoordinate=verts[2][2],
                         Vertex_4_Xcoordinate=verts[3][0], Vertex_4_Ycoordinate=verts[3][1], Vertex_4_Zcoordinate=verts[3][2])

    # Infiltration - Using correct field names
    idf.newidfobject("ZONEINFILTRATION:DESIGNFLOWRATE",
                     Name="MainZone Infiltration",
                     Zone_or_ZoneList_or_Space_or_SpaceList_Name="MainZone",
                     Schedule_Name="AlwaysOn",
                     Design_Flow_Rate_Calculation_Method="Flow/Area",
                     Design_Flow_Rate=0.0,
                     Flow_Rate_per_Floor_Area=0.0002,
                     Flow_Rate_per_Exterior_Surface_Area=0.0,
                     Air_Changes_per_Hour=0.0,
                     Constant_Term_Coefficient=1.0,
                     Temperature_Term_Coefficient=0.0,
                     Velocity_Term_Coefficient=0.0,
                     Velocity_Squared_Term_Coefficient=0.0)

    # Schedules
    idf.newidfobject("SCHEDULETYPELIMITS", Name="Fraction", Lower_Limit_Value=0.0, Upper_Limit_Value=1.0,
                     Numeric_Type="Continuous", Unit_Type="Dimensionless")

    idf.newidfobject("SCHEDULE:COMPACT", Name="AlwaysOn", Schedule_Type_Limits_Name="Fraction",
                     Field_1="Through: 12/31", Field_2="For: AllDays", Field_3="Until: 24:00, 1")

    # Remove thermostat control since we don't have HVAC system
    # This prevents the fatal error about thermostatic control without equipment connections
    
    # Just add basic output variables for thermal analysis
    for var in [
        "Zone Air Temperature",
        "Zone Mean Air Temperature",
        "Zone Infiltration Sensible Heat Gain",
        "Zone Infiltration Air Change Rate"
    ]:
        idf.newidfobject("OUTPUT:VARIABLE",
                         Variable_Name=var,
                         Reporting_Frequency="Hourly")

    # Write to file
    idf.save(idf_path)

In [70]:
def run_energyplus_simulation(home_dir, weather_path):
    """
    Runs EnergyPlus simulation from within the home_dir,
    using in.idf and a weather file located outside (e.g., in 'weather/').
    Outputs go to home_dir/simulation_output/.
    """
    original_cwd = os.getcwd()
    abs_weather_path = os.path.abspath(weather_path)  # Ensure weather path works across directories

    os.makedirs(home_dir, exist_ok=True)
    os.chdir(home_dir)

    try:
        # Run EnergyPlus with absolute weather path
        cmd = [
            "energyplus",
            "-w", abs_weather_path,
            "-d", "simulation_output",
            "-r",
            "in.idf"
        ]

        print(cmd)

        subprocess.run(cmd, check=True)
    finally:
        os.chdir(original_cwd)

Now, we define the function we'll use to simulate one home.

In [71]:
def simulate_home(home_folder_name: str):
    geojson_path = f"dataset/{home_folder_name}/cleaned.geojson"
    preprocessed_path = f"dataset/{home_folder_name}/preprocessed.json"

    with open(geojson_path, 'r') as f:
        geojson_data = json.load(f)
        generate_idf_from_geojson(geojson_data, f"dataset/{home_folder_name}/in.idf")

    with open(preprocessed_path, 'r') as f:
        preprocessed_data = json.load(f)
        weather_station = preprocessed_data["weather"]

    run_energyplus_simulation(
        f'dataset/{home_folder_name}',
        f'weather/{weather_station}.epw'
    )

    print(f"Completed simulation for {home_folder_name}.")

In [72]:
simulate_home('RAMBEAU_RD_15')

['energyplus', '-w', '/Users/test/Development/summer_2025/generating_data/weather/KABE.epw', '-d', 'simulation_output', '-r', 'in.idf']
EnergyPlus Starting
EnergyPlus, Version 25.1.0-68a4a7c774, YMD=2025.06.24 11:59
Adjusting Air System Sizing
Adjusting Standard 62.1 Ventilation Sizing
Initializing Simulation
Reporting Surfaces
Beginning Primary Simulation
Initializing New Environment Parameters
Warming up {1}
Warming up {2}
Warming up {3}
Warming up {4}
Warming up {5}
Warming up {6}
Starting Simulation at 06/24/2017 for RUNPERIOD1
Beginning Primary Simulation
Initializing New Environment Parameters
Warming up {1}
Warming up {2}
Warming up {3}
Warming up {4}
Warming up {5}
Warming up {6}
Starting Simulation at 06/24/2017 for RUNPERIOD1
 ReadVarsESO program starting.
 ReadVars Run Time=00hr 00min  0.01sec
 ReadVarsESO program completed successfully.
 ReadVarsESO program starting.
 Requested ESO file=simulation_output/eplusout.mtr
 does not exist.  ReadVarsESO program terminated.
 ReadVa

EnergyPlus Completed Successfully.


In [93]:
# TODO: Run this for all entries in the dataset

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/dataset/LENAIRE_RD_2/cleaned.geojson'